# Project 3 - SME808
* Leonardo Meireles - NUSP: 4182085
* Antonio Moreira - NUSP: 9779242

## Objective
* Dynamic Linear Models
* TS Final project

In [ ]:
# Basic packages
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd  # generating random numbers
import datetime  # manipulating date formats
# Viz
import matplotlib.pyplot as plt  # basic plotting
import matplotlib.dates as mdates
import seaborn as sns  # for prettier plots
import matplotlib.style as style
style.use('seaborn-white')

# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf, arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import scipy.special as sc
import pydlm

plt.rcParams.update({'font.size': 14})

# settings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def tsplot(y, lags=None, figsize=(14, 8), style='ggplot',title=''):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (3, 2)
        # Defining the subplot axes
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title(title)
        # Auto correlation plot(MA)
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        # Parcial ACF(AR)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)

        plt.tight_layout()
        plt.plot()

# House Property Sales Time Series
* Collected property sales data for the 2007-2019 period for one specific region. The data contains sales prices for houses and units with 1,2,3,4,5 bedrooms. These are the cross-depended variables.
* The other csv file contains the raw data but re-sampled at quarterly intervals using a median aggregator.
* The data can be summarised as:

    * date of sale
    * price
    * property type: unit or house
    * number of bedrooms: 1,2,3,4,5 (these are the multi-variables we are targeting)
    * 4digit postcode (this is for reference only, we don't expect that this data point will be useful)

* Goal is to predict the next 8 quartes with the lowest mse prediction error

* **Data taken from https://www.kaggle.com/htagholdings/property-sales** 

In [ ]:
df = pd.read_csv('../data/house_sales/raw_sales.csv')
df['data'] = pd.to_datetime(df.datesold)

In [ ]:
df = df[np.abs(df.price - df.price.mean()) <= (5.0 * df.price.std())] # Clean the outliers

In [ ]:
df = df[(df.propertyType=='house') & (df.bedrooms >= 2)]
df = df.drop(['postcode', 'propertyType', 'datesold'], axis=1)

In [ ]:
# # cleaning outliers
# df_clean = pd.DataFrame()
# #df = df[np.abs(df.price - df.price.mean()) <= (5.0 * df.price.std())] # Clean the outliers

# for n_bedrooms in df.bedrooms.unique():
#     df_b = df[df.bedrooms == n_bedrooms].copy()
    
#     df_b = df_b[np.abs(df_b.price - df_b.price.mean()) <= (5 * df_b.price.std())] # Clean the outliers
    
#     df_clean = df_clean.append(df_b, sort=True, ignore_index=True)
    
# df_clean.groupby([pd.Grouper(key='data', freq='1M'), 'bedrooms']).median()['price'].unstack().plot(figsize=(16, 5))

In [ ]:
df_clean.groupby([pd.Grouper(key='data', freq='3M'), 'bedrooms']).median()['price'].unstack().plot(figsize=(16, 5))

In [ ]:
df = pd.get_dummies(df, columns=['bedrooms'], prefix='n_quarto')

In [ ]:
df_monthly = df.groupby([pd.Grouper(key='data', freq='1M', closed='left')]).sum()

In [ ]:
# Set the locator
locator = mdates.MonthLocator()  # every month
# Specify the format - %b gives us Jan, Feb...
fmt = mdates.DateFormatter('%b')

diff_df = df_monthly.diff(1)

ax = df_monthly[(df_monthly.index.year) >= 2015].price.plot(figsize=(16, 5), x_compat=True)

#ax.set_xticklabels(xlabels, Rotation=2)
# set monthly locator
ax.xaxis.set_major_locator(locator)
# set formatter
ax.xaxis.set_major_formatter(fmt)
# set font and rotation for date tick labels
plt.gcf().autofmt_xdate()

ax.tick_params(axis='x', rotation=70)
plt.show()

In [ ]:
df_monthly.price.plot(figsize=(16, 5))

# Sazonalidade anual jan-fev sempre cai, periodo = 12
# Tendencia que muda a longo do tempo, degree = 1 (parece linear)

In [ ]:
time_series = df_monthly.price

In [ ]:
from pydlm import dlm, trend, seasonality
# A linear trend
linear_trend = trend(degree=1, discount=0.9, name='Tendencia Linear', w=5)
# A seasonality
yearly = seasonality(period=12, discount=0.98, name='Anual', w=5)
#quarter = seasonality(period=3, discount=0.90, name='Trimestral', w=10)

# Build a simple dlm
simple_dlm = dlm(time_series) + linear_trend + yearly# + quarter


In [ ]:
simple_dlm.fit()

# Plot each component (attribute the time series to each component)
simple_dlm.turnOn('predict plot')
simple_dlm.turnOff('filtered plot')
simple_dlm.turnOff('smoothed plot')

#simple_dlm.plot('linear_trend')
simple_dlm.plot()

In [ ]:
simple_dlm.plot('Tendencia Linear')
simple_dlm.plot('Anual')
#simple_dlm.plot('Trimestral')

In [ ]:
simple_dlm.getMSE()

In [ ]:
features = df_monthly[df_monthly.columns.difference(['price'])].values

In [ ]:
from pydlm import dynamic
regressor10 = dynamic(features=features, discount=1, name='regressor10', w=10)

drm = dlm(time_series) + linear_trend + yearly + regressor10
drm.fit()
drm.getMSE()

# Plot the fitted results
drm.turnOff('data points')
drm.plot()

In [ ]:
# Plot each component (attribute the time series to each component)
drm.turnOn('predict plot')
drm.turnOff('filtered plot')
drm.turnOff('smoothed plot')

#drm.plot('linear_trend')
drm.plot()


In [ ]:
drm.plotPredictN(date=100, N=49)